## Correct SQL Errors using Bedrock

In [17]:
# import boto3
# import botocore
# import langchain
# import json

In [18]:
#create db connection
# con = sqlite3.connect(DB_PATH)

In [19]:
# bedrock_runtime = boto3.client("bedrock-runtime")

In [20]:
# prompt_data = """\n\nHuman: Convert the following SQL statement in <postgres> tag from Postgres to SQLite compatible format.

# <postgres>
# {sql}
# </postgres>

# Provide answer in <sqlite> tag. Output only the answer in the tag. Nothing else.

# Assistant:<sqlite>
# """

# prompt_data = """\n\nHuman: Rewrite the following SQL statement to be executable in SQLite engine. The input SQL is provided in <postgres> tag.

# <postgres>
# {sql}
# </postgres>

# Provide answer in <sqlite> tag.

# Assistant:<sqlite>
# """

In [21]:
# def correct_errors(row, column):
#     """Correct the sql errors to those without from keyword."""
#     sql = row[column]
#     sql_out = sql
#     if 'from' not in sql.lower():
#         sql_out = sql.replace("where", "from beneficiary_summary where")
#     row[f'{column}_corrected'] = sql_out
#     return row


# def change_postgres_to_sqlite(row, column, prompt0):
#     """Convert the postgres to sqlite format."""
#     is_executable = row['query_executed']
#     sql = row[column]
#     if is_executable or ('diff' not in sql.lower()):
#         row[f'{column}_sqlite'] = sql
#         return row

#     prompt = prompt0.format(sql=sql)
#     print('Prompt:\n', prompt)

#     body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500, "temperature":0.0})
#     modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
#     accept = "application/json"
#     contentType = "application/json"

#     response = ""
#     try:

#         response = bedrock_runtime.invoke_model(
#             body=body, modelId=modelId, accept=accept, contentType=contentType
#         )
#         response_body = json.loads(response.get("body").read())

#         response = response_body.get("completion")

#     except botocore.exceptions.ClientError as error:

#         if error.response['Error']['Code'] == 'AccessDeniedException':
#                print(f"\x1b[41m{error.response['Error']['Message']}\
#                     \nTo troubeshoot this issue please refer to the following resources.\
#                      \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
#                      \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

#         else:
#             raise error
#     print('Original:', sql)
#     print('-'*30)
#     print('SQLite Version:', response)
#     print('*'*30)
#     row[f'{column}_sqlite'] = response
#     return row

In [22]:
# #Add missing `from` section
# df_qa = df_qa.apply(correct_errors, args=('query',), axis=1)
# df_qa.head()

In [23]:
# df_qa = df_qa.apply(change_postgres_to_sqlite, args=('query_corrected',prompt_data), axis=1)
# df_qa.head()

In [24]:
# column = 'query_corrected_sqlite'
# df_qa[df_qa[column].str.contains('<sqlite>')][column]

In [25]:
# column = 'query_corrected'
# print(f'Analysis for {column}\n{"="*50}')
# df_qa = df_qa.apply(run_df_row_query, args=(column, con), axis=1)

# df_qa[f'{column}_executed'].value_counts()

In [26]:
# df_qa.columns

In [1]:
import boto3

In [22]:
bedrock = boto3.client('bedrock')
# bedrock.list_foundation_models()

In [6]:
bedrock_runtime = boto3.client('bedrock-runtime')
bedrock_runtime

In [11]:
# If you'd like to try your own prompt, edit this parameter!
prompt_data = """Human: Write me a blog about making strong business decisions as a leader.

Assistant:
"""

In [12]:
import json
import botocore
body = json.dumps({"prompt": prompt_data, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-instant-v1"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

try:

    response = bedrock_runtime.invoke_model(
        body=body, modelId=modelId, accept=accept, contentType=contentType
    )
    response_body = json.loads(response.get("body").read())

    print(response_body.get("completion"))

except botocore.exceptions.ClientError as error:

    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")

    else:
        raise error



 Here is a draft blog post on making strong business decisions as a leader:

How to Make Strong Business Decisions as a Leader

Making difficult decisions is one of the most important responsibilities of being a leader. However, making decisions that effectively drive your business forward can be challenging. Here are some tips for leaders on how to make strong choices that will positively impact your business:

Consider all options carefully. Don't rush into a decision without properly weighing all potential courses of action. Take the time to thoroughly explore different options by considering risks, costs, potential obstacles, and opportunities associated with each. Run decisions by trusted advisors or form a small committee to gain different perspectives.

Focus on strategy, not just tactics. Make sure any decision ties back to and supports your overall business strategy and long-term goals. Don't get bogged down by short-term tactics if they detract from your strategic objectives.

In [26]:
bedrock_runtime = boto3.client('bedrock-runtime')

from langchain.llms.bedrock import Bedrock
def get_bedrock_llm(model_name):
    llm = Bedrock(client=bedrock_runtime,
                  model_id=MODEL_MAPPING[model_name], 
                  model_kwargs=GENERATOR_CONFIG[model_name])
    return llm

In [27]:
MODEL_MAPPING = {
    "Titan": "amazon.titan-tg1-large",
    "Claude": "anthropic.claude-v1",
    "Claude2": "anthropic.claude-v2",
    "Jurassic": "ai21.j2-jumbo-instruct",
    "Jurassic_ht": "ai21.j2-jumbo-instruct"    
}

GENERATOR_CONFIG = {
    "Titan": {
        "maxTokenCount": 512,
        "temperature": 0,
        "topP": 1,
    },
    "Claude": {
        "max_tokens_to_sample": 4096,
        "temperature": 0,
        "top_p": 1,
        "top_k": 50,
    },
    "Claude2": {
        "max_tokens_to_sample": 4096,
        "temperature": 0,
        "top_p": 1,
        "top_k": 50,
    },
    "Jurassic": {
        "maxTokens": 512,
        "numResults": 1,
        "temperature": 0,
        "topKReturn": 0,
        "topP": 1,
    },
}

In [28]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

model_name = "Claude2"
llm = get_bedrock_llm(model_name)
conversation = ConversationChain(
    llm=llm, verbose=True, memory=ConversationBufferMemory()
)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hello! Nice to meet you. What would you like to chat about today? I'm happy to have a friendly conversation about any topic."